In [1]:
import torch

# data loading
from plyfile import PlyData

# plotting
from meshplot import plot

# implementation
from implementation import point_rasterization, grid_interpolation, centerzoom_scaling

### Load data and compute rasterization

In [2]:
SCAN = "./deep_geometric_prior_data/scans/anchor.ply"
GRID = tuple([256] * 3)

In [3]:
def read_ply_file(file):
    data = PlyData.read(file)

    xyz = [torch.tensor(data["vertex"][axis]) for axis in ["x", "y", "z"]]
    xyz = torch.stack(xyz, dim=-1)

    nxyz = [torch.tensor(data["vertex"][axis]) for axis in ["nx", "ny", "nz"]]
    nxyz = torch.stack(nxyz, dim=-1)
    return xyz.unsqueeze(0), nxyz.unsqueeze(0)


from implementation import minmax_scaling


points, normals = read_ply_file(SCAN)
points = torch.sigmoid(centerzoom_scaling(points, scale=0.9))

In [4]:
# encode normals into grid
normal_vector_field = point_rasterization(points, normals, GRID)

In [5]:
# retrieve normal values
# permute: (one grid per feature) -> (grid with all features)
normal_vector_field = normal_vector_field.permute(0, 2, 3, 4, 1)
n = grid_interpolation(normal_vector_field, points)

### Example Rasterization
Following we show the difference between interpolated normals and true normals

In [6]:
# plot with interpolated normals
plot(points[0].numpy(), c=n[0].numpy())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4627328…

In [7]:
# plot with actual normals
plot(points[0].numpy(), c=normals[0].numpy())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4627328…